In [ ]:
!pip install -q langchain langchain_community langchain_chroma langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-1.5-pro")

In [ ]:
!pip install -q pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.9 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/2306.05685")
pages = loader.load_and_split()

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=pages, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [ ]:
retriever = vectorstore.as_retriever()

In [13]:
prompt = """
Based on the following initial prompt, generate five optimal prompts that vary in perspective, detail level, scope, and incorporate different elements to stimulate diverse responses. The initial prompt is: [Insert Initial Prompt Here].

1. Write a prompt that focuses on the same theme but from a first-person perspective.
2. Create a prompt that explores a more specific aspect of the original theme.
3. Develop a prompt that broadens the scope to include related topics or wider contexts.
4. Construct a prompt that includes a unique constraint or element to add complexity.
5. Formulate a prompt that emphasizes detailed descriptions and sensory details.

Ensure each new prompt maintains the core theme of the initial prompt while offering a unique angle or approach.
Just list the proposed prompts in a comma-separated list.
{initial_prompt}
"""

from langchain_core.prompts import PromptTemplate
optimal_prompts_prompt = PromptTemplate.from_template(prompt)

chain_generation_prompts =  optimal_prompts_prompt | llm
result = chain_generation_prompts.invoke("What is LLM-as-a-judge?")

In [14]:
result

'1. As a large language model, how would I feel if I were tasked with being the ultimate judge in a court of law, weighing evidence and delivering verdicts?, 2. What are the ethical implications of using a large language model to determine the outcome of legal cases, particularly in terms of bias and fairness?, 3. How might the advent of LLM-as-a-judge impact the future of the legal profession and the justice system as a whole, considering its potential benefits and drawbacks?, 4. Imagine a world where an LLM judge must decide a case with no legal precedent, relying solely on its understanding of morality and ethics. How might it rule, and what challenges would it face?, 5. Describe the scene in a futuristic courtroom where an LLM judge, with its vast knowledge and analytical capabilities, presides over a complex trial, focusing on the sights, sounds, and atmosphere. \n'

In [21]:
from langchain_core.runnables import RunnablePassthrough

rank_prompts_prompt = """
Given the context, evaluate and rank the following prompts in order of their effectiveness in addressing the context. Consider factors such as relevance, clarity, creativity, and ability to engage the intended audience.

Context: {context}

Prompts:
{optimal_prompts}

Identify and provide only the best prompt based on this evaluation. Don't provide a justification.
"""

ranked_prompt_template = PromptTemplate.from_template(rank_prompts_prompt)
chain_rank_prompts = (
    {"context": retriever, "optimal_prompts": RunnablePassthrough()}
    | ranked_prompt_template
    | llm
)
result2 = chain_rank_prompts.invoke(result)

In [22]:
result2

'3. How might the advent of LLM-as-a-judge impact the future of the legal profession and the justice system as a whole, considering its potential benefits and drawbacks? \n'

In [29]:
qa_prompt = """
Given the following context, generate a comprehensive and well-informed response to the prompt provided. Ensure that the response is grounded in the latest 2024 scientific literature, and consider dimensions such as relevance, clarity, depth, creativity, accuracy, and engagement. Incorporate up-to-date research, statistics, and expert opinions to enhance the quality and reliability of the response.

Context: {context}

Prompt: {ranked_prompt}

Key Considerations:
1. **Relevance**: Ensure the response is directly related to the context and prompt.
2. **Clarity**: Present the information in a clear, concise, and understandable manner.
3. **Depth**: Provide thorough and detailed information, covering all necessary aspects.
4. **Creativity**: Include innovative and original ideas where appropriate.
5. **Accuracy**: Base the response on current, verified scientific literature and data from 2024.
6. **Engagement**: Make the response engaging and compelling for the intended audience.

Use these considerations to produce an optimal response that effectively addresses the prompt within the given context.
"""

qa_prompt_template = PromptTemplate.from_template(qa_prompt)
chain_qa = (
    {"context": retriever, "ranked_prompt": RunnablePassthrough()}
    | qa_prompt_template
    | llm
)
result3 = chain_qa.invoke(result2)

In [30]:
result3

'## The Gavel and the Algorithm: How LLM-as-a-Judge Could Reshape the Legal Landscape\n\nThe emergence of Large Language Models (LLMs) as potential judges, as explored in "Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena" (Zheng et al., 2023), presents both intriguing possibilities and complex challenges for the legal profession and the justice system. While still in its nascent stages, this technology has the potential to revolutionize legal proceedings, but not without careful consideration of its implications.\n\n**Potential Benefits:**\n\n* **Enhanced Efficiency and Accessibility:**  LLM-judges could automate many tasks currently performed by human judges, such as legal research, document review, and even preliminary case assessments. This could significantly expedite legal processes, reduce backlogs, and potentially make legal services more affordable and accessible to a wider population. \n* **Increased Objectivity and Consistency:** One of the purported advantages of LLMs 

In [33]:
full_chain = chain_generation_prompts | chain_rank_prompts | chain_qa
final_result = full_chain.invoke("What is LLM-as-a-judge?")

In [34]:
final_result

"## Using LLMs to Enhance Fairness and Accuracy in Judging Subjective Competitions\n\nWhile the document focuses on evaluating LLMs' ability to judge open-ended responses, it hints at a crucial application: using this technology to improve fairness and accuracy in judging subjective competitions like essay writing or art contests. Here's how LLMs can be leveraged, along with potential challenges:\n\n**1. Objective Metric Extraction:**\n\n* **LLMs can analyze subjective content and extract quantifiable metrics.** For essays, this could include vocabulary complexity, argument structure, originality of ideas, and adherence to grammar and style guidelines. For art, metrics could involve color palettes, composition techniques, emotional impact based on image analysis, and adherence to specific styles or themes.\n* **Example:** An LLM could be trained on a dataset of highly-rated essays to identify recurring patterns and characteristics that contribute to a strong piece, providing a score ba

In [35]:
full_chain = chain_generation_prompts | chain_rank_prompts | chain_qa
final_result = full_chain.invoke("What is LLM-as-a-judge?")

In [36]:
final_result

'## The Gavel and the Algorithm: Weighing the Potential of LLMs as Judges\n\nThe prospect of using LLMs as judges in the justice system, while seemingly futuristic, presents both exciting opportunities and significant challenges.  Let\'s examine the potential benefits and drawbacks through the lens of access to justice, bias mitigation, and public trust:\n\n**Potential Benefits:**\n\n* **Increased Access to Justice:** \n    * **Speed and Efficiency:** LLMs could process cases significantly faster than human judges, potentially alleviating court backlogs and providing quicker resolutions.  This could be particularly impactful in areas with limited legal resources, providing underrepresented communities with faster access to legal decisions.  \n    * **Cost Reduction:**  The automation provided by LLMs could lower the cost of legal proceedings, making legal representation more accessible to individuals and small businesses who might otherwise struggle to afford it.\n* **Mitigating Human 